In [1]:
import pandas as pd
import numpy as np
import os
raw_data = pd.read_csv(os.path.join('..', 'data', 'Restroke_SPSS.csv'))
raw_data.replace({999.0: np.nan}, inplace=True)

In [2]:
print('Subjects:{}, Features:{}'.format(raw_data.shape[0], raw_data.shape[1]))

Subjects:7870, Features:62


In [3]:
raw_data.head()

,CHT_No,Sex,Age,HTN,DM,Dyslipidemia,AF,CVA_Hx,CT_Old_Lesion,Smoking,...,CT_0ew_Lesio0,Alcohol,Adm_AF_0otEKG,EKG_AF,BU0-Cr Ratio,BUN-Cr,Glucose,NIHSS_Dis,BI_Dis,MRS_Dis
0,20290.0,0.0,74.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,100.0,1
1,23614.0,0.0,87.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,16.14,0.0,94.0,1.0,100.0,1
2,24832.0,0.0,65.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,NaN,NaN,84.0,2.0,100.0,1
3,34018.0,0.0,75.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,19.63,0.0,NaN,2.0,100.0,1
4,45874.0,0.0,84.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,28.57,1.0,133.0,31.0,0.0,5


In [4]:
print('Age_mean: %.2f, Age_std: %.2f,' %(np.mean(raw_data.Age), np.std(raw_data.Age)))
male_protion = (raw_data[raw_data.Sex==1].shape[0]/raw_data.Sex.shape[0])*100
print('Male:%.2f%%' %male_protion)
restroke_protion = (raw_data[raw_data.reStroke==1].shape[0]/raw_data.reStroke.shape[0])*100
print('reStroke:%.2f%%' %restroke_protion)

Age_mean: 67.75, Age_std: 12.74,
Male:62.48%
reStroke:8.58%
